# $\color{teal}{\text{4.3 Iterative Hard Thresholding}}$
The so-called restricted isometry property (RIP) of matrices gives a hint on whether the sparse recovery algorithm will succeed or not. It is NP-hard to show that a given matrix fullfilles the RIP, but fortunately a theorem assures that subgaussian random matrices with i.i.d. entries satisfy the RIP with high probability. Therefore, we are going to use the numpy function `random.randn()` to generate the corresponding measurement matrix $A$.


The iterative hard thresholding (IHT) algorithm is based on non-convex optimization. Hence, it converges to local minima and depends on the initialization of `x_0` (commomly with the all-zero vector). In following we will implement the general setup of CS, i.e. generate the measurement matrix $A$ and the measurements $y$ based on the true vector `x_true` that is randomly generated. 

In [ ]:
import numpy as np
import random
import math


#iht:
m = 300
N = 1000
s = 10

#omp
#m = 200
#N = 175
#s = 30

#ground truth
x_true = np.zeros(N)
indices = random.sample(range(N),s) #randomly sample s out of 1000 indices
x_true[indices] = np.random.rand(s)

#random measurement matrix
A = np.random.randn(m,N)
A = A/math.sqrt(m)
#simulated measurements
y = A @ x_true

In each iteration of the IHT algorithm we calculate 

$$x^{(k+1)} = \mathcal{H}_s\{x^{(k)} + A^{*}(y-Ax^{(k)})\}$$

where $\mathcal{H}_s$ denotes the hard thresholding operator of order $s$ and $A^{*}$ the hermitian of $A$. By applying $\mathcal{H}_s$ we take only the $s$ entries with the largest absolute values and set the other ones to zero, thus ensuring the sparsity $s$ of $x^{(k+1)}$. 

In [ ]:
def H_s(x_tmp,s):
    """Hard-thresholding operator: keep the s largest entries of z and set the other ones to zero."""
    N = x_tmp.shape[0]
    x_new = np.zeros(N)
    indices = x_tmp.argsort()[-s:]
    x_new[indices] = x_tmp[indices]
    return x_new

In [ ]:
def iht(A, y, s, iters):
    """Implementation of the IHT."""
    [m,N] = A.shape
    x_0 = np.zeros(N)
    
    #hermitian of A
    A_H = A.conjugate().transpose()
    
    x_iters = []
    err_iters = np.zeros(iters)
    x_hat = np.zeros(N)
    
    from tqdm import tqdm
    for i in tqdm(range(iters)):
        if i==0:
            x_k = x_0
        else:
            x_k = H_s(x_k + A_H@(y - A@x_k),s)
            #@TODO: iteratively plot x_k and x_true
            #print(f"x_hat_{i}: {[val for val in x_k if val!=0]}\n")
            x_hat = x_k
        x_iters.append(x_k)
        
        err = np.linalg.norm(A@x_k-y,2) #l2-error
        err_iters[i] = err
        print(f"err iteration {i}: {err}")
    return x_hat, x_iters, err_iters

Run the iterative hard thresholding algorithm.

In [ ]:
n_iters = 100
x_hat, x_iters, err_iters = iht(B,z,10,n_iters)

In [ ]:
err = np.linalg.norm(x_hat-x_true,2) #l2-error
print(err)